In [ ]:
%pip install pandas

In [1]:
from datetime import datetime
from collections import defaultdict

import pandas as pd

from ip_as_org import IPASnPrefix, ASOrg

In [2]:
def get_ip_asn_as_org_obj(timestamps):
    dates = [ts.strftime("%Y%m%d") for ts in timestamps]
    dataset_dir = "../dataset/"
    ip_asn = IPASnPrefix(dates, dataset_dir)
    as_org = ASOrg(dates, dataset_dir)
    return ip_asn, as_org


def get_nr_prefixes_from_asn_list(asn_list, ip_asn):
    prefixes_dict = defaultdict(int)
    for asn in asn_list:
        if asn is None:
            continue
        prefixes = ip_asn.get_prefixes_from_asn(asn, when)
        if prefixes is None:
            continue
        for prefix in prefixes:
            if prefix is None:
                continue
            prefix = prefix.split("/")[1]
            prefixes_dict[prefix] += 1
    return prefixes_dict

def get_distr_prefix_len(prefixes):
    prefixes_dict = defaultdict(int)
    for prefix in prefixes:
        if prefix is None:
            continue
        prefix = prefix.split("/")[1]
        prefixes_dict[prefix] += 1
    return prefixes_dict

def get_nr_ips_per_prefix(prefixes_dict):
    nr_ips = 0
    for prefix_len in prefixes_dict.keys():
        e = 32 - int(prefix_len, 10)
        nr_ips += 2 ** e - 2  # network address and default gw
    return nr_ips

ip_asn, as_org = get_ip_asn_as_org_obj([
    datetime(2024,7,9),
    datetime(2024,7,10),
    datetime(2024,7,11),
    datetime(2024,7,12),
])

when = "20240709"
org_list = ["Google", "Hetzner", "OVH", "NhanHoa", "Xneelo", "Contabo", "DigitalOcean"]
amazon = ["AWS", "Amazon"]
microsoft = ["Azure", "Microsoft"]
ali = ["Aliyun", "Alibaba"]

Extract nr of IPs each provider has

In [3]:
org_pdf = pd.read_csv("output/cloud_providers.csv")

In [4]:
def get_nr_ips_per_org(org_name):
    asns = as_org.get_asn_from_org_name(org_name, when)
    prefixes_dict = get_nr_prefixes_from_asn_list(asns, ip_asn)
    nr_ips = get_nr_ips_per_prefix(prefixes_dict)
    return nr_ips

org_pdf["nr_ips"] = org_pdf["cloud_provider"].apply(get_nr_ips_per_org)

In [11]:
print("% of not found/no IP address", round(len(org_pdf[org_pdf["nr_ips"] == 0]) / len(org_pdf) * 100, 1))

org_pdf[org_pdf["nr_ips"] > 0].sort_values("nr_ips", ascending=False)

% of not found/no IP address 97.2


,cloud_provider,nr_ips
3905,Level,12582850
1756,Internet,4194145
4324,Taiwan,4194082
1002,Beltelecom,1048296
2484,GTT,524228
...,...,...
1295,DataXion,254
1441,Nodisto,254
3509,EboundHost.com,254
1832,Withsystems,254


In [73]:
org_pdf.to_csv("output/cloud_providers_size.csv", index=False)

In [9]:
org_name = "ROU"
asns = as_org.get_asn_from_org_name(org_name, when)
print(asns)
prefixes_dict = get_nr_prefixes_from_asn_list(asns, ip_asn)

as_org.get_org_name_from_asn(35357, when)

[]


'N&TS GROUP NETWORKS & TRANSACTIONAL SYSTEMS GROUP S.P.A.'

Known providers

In [45]:
for org in org_list:
    asns = as_org.get_asn_from_org_name(org, when)
    prefixes_dict = get_nr_prefixes_from_asn_list(asns, ip_asn)
    nr_ips = get_nr_ips_per_prefix(prefixes_dict)
    print(org, "has", nr_ips, "IPs")


prefixes_dict = defaultdict(int)
for org in amazon:
    asns = as_org.get_asn_from_org_name(org, when)
    prefixes_dict.update(get_nr_prefixes_from_asn_list(asns, ip_asn))
nr_ips = get_nr_ips_per_prefix(prefixes_dict)
print("Amazon + AWS have", nr_ips, "IPs")


prefixes_dict = defaultdict(int)
for org in microsoft:
    asns = as_org.get_asn_from_org_name(org, when)
    prefixes_dict.update(get_nr_prefixes_from_asn_list(asns, ip_asn))
nr_ips = get_nr_ips_per_prefix(prefixes_dict)
print("Microsoft + Azure have", nr_ips, "IPs")


prefixes_dict = defaultdict(int)
for org in ali:
    asns = as_org.get_asn_from_org_name(org, when)
    prefixes_dict.update(get_nr_prefixes_from_asn_list(asns, ip_asn))
nr_ips = get_nr_ips_per_prefix(prefixes_dict)
print("Aliyun + Alibaba have", nr_ips, "IPs")

Google has 4194020 IPs
Hetzner has 257774 IPs
OVH has 261868 IPs
NhanHoa has 1786 IPs
Xneelo has 65264 IPs
Contabo has 32498 IPs
DigitalOcean has 65264 IPs
Amazon + AWS have 4194020 IPs
Microsoft + Azure have 8388322 IPs
Aliyun + Alibaba have 1048295 IPs


In [54]:
google_cloud_prefixes = pd.read_json("cloud-ip-prefixes/ip_prefixes/google/cloud.json")
ipv4_prefixes = google_cloud_prefixes["prefixes"].apply(lambda x: x.get("ipv4Prefix"))
prefixes_dict = get_distr_prefix_len(ipv4_prefixes.to_list())
nr_ips = get_nr_ips_per_prefix(prefixes_dict)
print("Google Cloud has", nr_ips, "IPs")

Google Cloud has 524010 IPs


In [35]:
for org in org_list:
    print(org)
    asns = as_org.get_asn_from_org_name(org, when)
    prefixes_dict = get_nr_prefixes_from_asn_list(asns, ip_asn)
    display(prefixes_dict)

Google


defaultdict(int,
            {'19': 132,
             '21': 99,
             '24': 1224,
             '18': 44,
             '16': 91,
             '17': 72,
             '22': 65,
             '20': 2031,
             '14': 31,
             '23': 58,
             '13': 10,
             '15': 9,
             '12': 5,
             '11': 3})

Hetzner


defaultdict(int,
            {'16': 31,
             '24': 169,
             '18': 2,
             '23': 9,
             '22': 12,
             '17': 6,
             '19': 1,
             '15': 1,
             '21': 2})

OVH


...

defaultdict(int,
            {'21': 4,
             '24': 18,
             '20': 6,
             '22': 9,
             '23': 3,
             '17': 2,
             '19': 2,
             '18': 1})

Contabo


defaultdict(int,
            {'20': 18,
             '22': 27,
             '21': 34,
             '24': 155,
             '23': 307,
             '18': 1,
             '19': 5})

DigitalOcean


defaultdict(int,
            {'24': 51,
             '22': 167,
             '20': 458,
             '23': 6,
             '18': 21,
             '19': 23,
             '21': 18,
             '17': 4})

In [37]:
print("Amazon + AWS")
prefixes_dict = defaultdict(int)
for org in amazon:
    asns = as_org.get_asn_from_org_name(org, when)
    prefixes_dict.update(get_nr_prefixes_from_asn_list(asns, ip_asn))
display(prefixes_dict)

print("Microsoft + Azure")
prefixes_dict = defaultdict(int)
for org in microsoft:
    asns = as_org.get_asn_from_org_name(org, when)
    prefixes_dict.update(get_nr_prefixes_from_asn_list(asns, ip_asn))
display(prefixes_dict)

print("Aliyun + Alibaba")
prefixes_dict = defaultdict(int)
for org in ali:
    asns = as_org.get_asn_from_org_name(org, when)
    prefixes_dict.update(get_nr_prefixes_from_asn_list(asns, ip_asn))
display(prefixes_dict)

Amazon + AWS


defaultdict(int,
            {'20': 158,
             '19': 59,
             '22': 784,
             '24': 5655,
             '23': 837,
             '21': 593,
             '16': 116,
             '15': 107,
             '17': 90,
             '18': 52,
             '14': 43,
             '12': 8,
             '13': 13,
             '11': 2})

Microsoft + Azure


defaultdict(int,
            {'24': 417,
             '23': 47,
             '20': 14,
             '22': 26,
             '21': 15,
             '16': 58,
             '19': 13,
             '15': 26,
             '12': 11,
             '17': 16,
             '11': 5,
             '18': 10,
             '14': 21,
             '10': 2,
             '13': 11})

Aliyun + Alibaba


defaultdict(int,
            {'24': 463,
             '23': 124,
             '32': 5,
             '22': 83,
             '20': 52,
             '16': 99,
             '17': 134,
             '14': 13,
             '21': 63,
             '15': 33,
             '18': 91,
             '19': 75,
             '13': 1})